In [1]:
import torch
import os
from PIL import Image
import torchvision.transforms as T
import pandas as pd
from rle import rle_decode
import numpy as np
from tqdm import tqdm

torch_ver_major = int(torch.__version__.split('.')[0])
dtype_index = torch.int32 if torch_ver_major >= 2 else torch.long

train_dir = "/kaggle/input/blood-vessel-segmentation/train/"
msks_dir = f"{train_dir}kidney_1_dense/labels/"
imgs_dir = f"{train_dir}kidney_1_dense/images/"
slices_ids = sorted(os.listdir(imgs_dir))
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
w = Image.open(msks_dir + slices_ids[0]).width
h = Image.open(msks_dir + slices_ids[0]).height
print("Width:", w)
print("Height:", h)

Width: 912
Height: 1303


In [3]:
class TestMetricDataset(torch.utils.data.Dataset):
    def __init__(self, sub_df, msks_dir, slices_ids, transform=None, target_transform=None):
        self.sub_df = sub_df
        self.msks_dir = msks_dir
        self.slices_ids = slices_ids
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.slices_ids)
    
    def __getitem__(self, idx):
        slice_id = self.slices_ids[idx]

        pred_rle = sub_df.iloc[idx]["rle"]
        pred = rle_decode(pred_rle, (h, w))
        pred = torch.from_numpy(pred)

        target_path = self.msks_dir + slice_id 
        target = Image.open(target_path)

        if self.target_transform is not None:
            target = self.target_transform(target).type(torch.int8).squeeze()

        return pred, target

In [4]:
sub_df = pd.read_csv("ref_sub.csv")

target_transform = T.Compose([
    T.ToTensor(), 
])

ds = TestMetricDataset(sub_df=sub_df, msks_dir=msks_dir, slices_ids=slices_ids, target_transform=target_transform)
# batch_size needs to be an odd number
dl = torch.utils.data.DataLoader(ds, batch_size=3, num_workers=os.cpu_count(), drop_last=False)
n_batches = len(dl)
print("ds len:", len(ds))
print("dl len:", n_batches)


ds len: 2279
dl len: 760


In [5]:
from surface_dice import SurfaceDiceMetric
device = "cpu"
metric = SurfaceDiceMetric(n_batches, device)
c = 1
for pred, target in dl:
    print(f"{c}/{len(dl)}")
    c += 1
    pred, target = pred.to(device), target.to(device)
    metric.process_batch(pred, target)
metric.compute_metric()

# bs 1
# 12641531.0 14366878.0
# 0.87990802526474
 
# bs 3
# 12641525.0 14366888.0
# 0.879906952381134
 
# bs 5
# 12641491.0 14366864.0
# 0.8799060583114624
# 0.8799066326898859

float64
1/760


True
True
2/760
True
True
3/760
True
True
4/760
True
True
5/760
True
True
6/760
True
True
7/760
True
True
8/760
True
True
9/760
True
True
10/760
True
True
11/760
True
True
12/760
True
denominator 3.552713678800501e-15
13/760
True
True
14/760
True
True
15/760
True
True
16/760
True
True
17/760
True
True
18/760
True
denominator 1.7763568394002505e-15
19/760
True
True
20/760
numerator 1.4210854715202004e-14
True
21/760
True
True
22/760
True
True
23/760
numerator 7.105427357601002e-15
True
24/760
numerator 1.4210854715202004e-14
True
25/760
numerator 2.842170943040401e-14
True
26/760
numerator 2.842170943040401e-14
True
27/760
True
True
28/760
True
True
29/760
numerator 5.684341886080802e-14
True
30/760
numerator 5.684341886080802e-14
denominator 1.1368683772161603e-13
31/760
True
True
32/760
numerator 1.1368683772161603e-13
denominator 1.1368683772161603e-13
33/760
True
True
34/760
True
denominator 2.2737367544323206e-13
35/760
numerator 2.2737367544323206e-13
True
36/760
numerator 2.27373

0.8799066326898859